## Deep learning: Chapter 2


Example:  Classify handwritten digits

* grayscale images, 28x28 pixels.  Sort these into 10 categories (digits 0 to 9)
* [MNISt](https://en.wikipedia.org/wiki/MNIST_database) dataset: 60k training images, 10k test images


Terminology:

* **Class**: a category in a classification problem 
* **Sample**: datapoints
* **Label**: class associatied with a specific sample

MNIST database is preloaded in KERASin set of four Numpy arrays:

In [1]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Using TensorFlow backend.
11493376/11490434 [==============================] - 3s 0us/step


* `train_images` and `train_labels` from the training set
* `test_images` and `test_labels` is test set on which model will be tested

In [2]:
print(f"Shape train_image = {train_images.shape}")
print(f"Length train_labels = {len(train_labels)}")
print(f"Shape test_images= {test_images.shape}")
print(f"Length test_labels = {len(test_labels)}")

Shape train_image = (60000, 28, 28)
Length train_labels = 60000
Shape test_images= (10000, 28, 28)
Length test_labels = 10000


### Workflow: 

1. feed neuronal network training data
2. associate images with labels
3. produce predictions for test images
4. verify whether predictions match test labels

In [3]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation = 'relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation = 'softmax'))

* layer: is a filter
* layers extract representations of data fed into them.
* layers are chained

Network above consists of a sequence of two dense layers that are *densely* connected (or *fully* connected) neural layers. The second layer is a 10 way *softmax* layer which returns 10 probability scores (summing to 1). Each score will give probability that current digit belongs to one of the 10 digit classes.

To make the network ready for training need to pick three more things as part of the *compilation* step: 

* an optimizer (method to update network)
* a loss function (how network measures performance on training data)
* metrics to monitor during training and testing (here: accuracy of classification)

A representation of the work flow is below:

<img src="https://dpzbhybb2pdcj.cloudfront.net/chollet/Figures/01fig09.jpg" width=400>

In [4]:
network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

* reshape images into a `float32` array of shape (60000, 28*28)
* rescale 8 bit images into values in interval `[0, 1]`

In [5]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32')/255

test_images = test_images.reshape((10000, 28 *28))
test_images = test_images.astype('float32')/255

* categorically encode labels:

In [6]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

* train network.  Call network's fit method:

In [7]:
network.fit(train_images, train_labels, epochs =5, batch_size=128)


Epoch 1/5
60000/60000 [==============================] - 2s 37us/step - loss: 0.2544 - accuracy: 0.9261
Epoch 2/5
60000/60000 [==============================] - 2s 33us/step - loss: 0.1046 - accuracy: 0.9695
Epoch 3/5
60000/60000 [==============================] - 2s 36us/step - loss: 0.0699 - accuracy: 0.9793
Epoch 4/5
60000/60000 [==============================] - 2s 33us/step - loss: 0.0513 - accuracy: 0.9843
Epoch 5/5
60000/60000 [==============================] - 2s 33us/step - loss: 0.0386 - accuracy: 0.9884


* evaluate how good network is at categorising unknown data:

In [8]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_acc:', test_acc)


10000/10000 [==============================] - 0s 29us/step
test_acc: 0.9787999987602234


This is lower than the training set accuracy.  This is an example of *overfitting*. 

This is in the workflow in short for generating a DL network in Keras. In what follows, we discuss what's behind the working.  We'll discuss data, tensors, tensor operations, data transoformations.

The types of data we are dealing with are:

* Vector data: e.g. dataset of text documents where each document is represented by the counts of how many times each word in a dictionary appears in it.  e.g. tensor of shape `(500, 20000)` might describe 500 documents and the count for each of the words contained in a dictionary of 20000 words.

* Time series data: e.g.stock prices as a tensor of rank 3: 250 days with values recorded every minute (390 working minutes per day), recording low, high and current price for each minute `(250, 390, 3)`

* Image data: tensor of rank 4: e.g. 128 images, 256 x 256 pixels, 3 colour channels. `(128, 256, 256, 3)`. note Tensorflow convention for order is `(samples, height, width, color_depth)` 

### Tensor attributes

* Number of axes (rank)
* shape: dimension of tensor along each axis
* Data type (usually `dtype` in python libraries)

In [9]:
print('shape of train_images = ', train_images.shape)  #Note: above we have vectorised 28x28 images

shape of train_images =  (60000, 784)


In [10]:
import numpy as np

In [11]:
y = np.reshape(train_images, (60000, 28,28)) #reshape into tensor of rank 3 (sample, height, width)
print('shape of y =', y.shape)

shape of y = (60000, 28, 28)


In [12]:
import matplotlib.pyplot as plt

In [13]:
digit = y[3]
digit.shape

(28, 28)

In [14]:
plt.imshow(digit)
plt.show()

In [ ]:
digit = y[34001]
plt.imshow(digit, cmap=plt.cm.binary)
plt.show()

NameError: name 'y' is not defined

### Slicing tensors:

Slicing just bottom half of 8:

In [2]:
digit = y[34001, 15:, :]
plt.imshow(digit, cmap=plt.cm.binary)
plt.show()

NameError: name 'y' is not defined

In [3]:
counter=0
start_index = 1011
tiles = 7
images = y[start_index:start_index+tiles]


fig=plt.figure()
for i in range(1,len(images)+1):
    sub = fig.add_subplot(1, len(images), i)
    sub.axis('off')
    sub.imshow(images[i-1])
    
plt.show()

NameError: name 'y' is not defined

### Data batches

the first axes (axis 0) is called the **sample axis**.  Usually the entire dataset can't be processed at once and one needs to break the dataset into smaller **batches**.  

In [4]:
batch_1 = train_images[:128]

batch_4 = train_images[128*4:128*(4+1)]  #batch n is from dataset[128*n:128*(n+1)] 

len(batch_4)

NameError: name 'train_images' is not defined

## Tensor operations

All transformations learned by DNN can be reduced to a small number of tensor operations (add, multiply, etc)

E.g.
```python
network.add(layers.Dense(512, activation = 'relu', input_shape=(28 * 28,)))
```

this layer cab be interpreted as a function that takes a 2D tensor (vecor samples) and returns another 2D tensor.

The function is:

```python
output = relu(dot(W, input) + b
```

* This includes a dot product between a 2D tensor W and an input tensor and adds a vector b. 
* It also includes a **rectified linear unit** function defined as `max(x, 0)`

### Rectified linear Unit - relu

In [5]:
def naive_relu(x):
    '''Naive implementation of relu (rectified linear unit) function
    takes tensor x of rank 2
    ''' 
    assert len(x.shape) == 2, 'x has more than 2 axes, not an image!'
    x = x.copy()    #produce a copy of x without overwriting input tensor
   
    
    for i in range(0, x.shape[0]):
        for j in range(0, x.shape[1]):
            x[i, j]=max(x[i,j], 0)
    return x

In [6]:
x = 2.0*(np.random.random((3,4))-0.5)  #random.random only produces positive numbers in (0, 1)

print(f"x=\n{x} \n")
print(f"Relu(x) = naive_relu(x) =\n{naive_relu(x)}")

NameError: name 'np' is not defined

same is achieved with the very simple inbuilt numpy method:

In [7]:
print(f"Relu(x) = np.maximum(x, 0) =\n{np.maximum(x,0)}")

NameError: name 'np' is not defined

## naive Addition

In [8]:
def naive_addition(x, y):
    assert len(x.shape)== 2, "Shape of x or y not rank 2!"
    assert x.shape == y.shape, "Shape of x and y not same!"
    
    x=x.copy()
    y=y.copy()
    
    for i in range(0, x.shape[0]):
        for j in range(0, x.shape[1]):
            x[i,j]+=y[i,j]
            
    return x

In [9]:
a = np.random.randint(5, size=(2,3))
print(f"a =\n{a}")

b = np.random.randint(5, size=(2,3))
print(f"b =\n{b}")

print(f"a + b = naive_addition(a, b) =\n{naive_addition(a, b)}")

NameError: name 'np' is not defined

### Broadcasting

Means stretching dimensions of lower rank tensor to those of higher rank tensor for efficient computation.

See [broadcasting article](https://numpy.org/devdocs/user/theory.broadcasting.html)

* works from last axis backward to front
* final dimensions must match

E.g. adding `1` to `[1,2,3]` is the same as broadcasting `1` to `[1,1,1]` and adding `[1,1,1]` to `[1,2,3]`.

In [10]:
import numpy as np

In [11]:
a = np.arange(1,4)
print(a)


[1 2 3]


In [12]:
x = np.arange(1,11).astype(float)
y = np.ones((2, 3, 10)).astype(float)
y[:]=x
print('y=\n',y)

print('\nrank of y:', y.ndim)

print('\nshape of y:', y.shape)

y=
 [[[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
  [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
  [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]]

 [[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
  [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
  [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]]]

rank of y: 3

shape of y: (2, 3, 10)


## Gradient-based optimisation

each neural layer transforms input data as follows:

```
output = relu(dot(W, input) +b)
```

where W and b are tensors that are attributes of layers, called weights (*kernel*) or trainable parameters (*bias*) of the layer.  Initially, this does nothing useful, but gradually W and b are adjusted, based on feedback signal.  This adjustment takes place in a *training loop*.  Specifically

1. Draw batch of training samples `x` and corresponding targets `y`
1. Run network on `x` (called *forward pass*) to obain `y_predicted`
1. Compute loss of network on batch, a measure of mismatch between `y_predicted` and `y`
1. Update  weights of `W`, `b` so that loss is reduced. 

Repeat, until loss is low: The network has now learned to map inputs to their correct targets.  
* Step 1 is simple: just I/O code.
* Steps 2 and 3 are merely application of a handful of tensor ops.
* Step 4 is the difficult part.  How do we decide whether `W` and `b` coefficients should be increased or decreased? And by how much?


Could do latter by freezing all coefficients except one, which you vary and recalculate loss.  If loss greater, change coefficient accordingly, etc.  Repeat sequentially for all coefficients. Problem: Terribly inefficient! There may be 10s or even 100s of thousands of coefficients to optimise: this makes problem intractable.  However: all operations described are differentiable. Better way: compute **gradient** of the loss wih regards to network coefficients

**Derivative:**

Approximate `f` around a point `p` as:

`f(x + epsilon_x) = y + a * epsilon_x`

The slope `a` is called the derivative of `f` in `p`

## Tensor multiplication

### vector dot product


$$ z = \vec{x} \cdot \vec{y}$$

where $z$ is a scalar

In [13]:
def naive_vector_dot(x,y):
    assert len(x.shape)==1, "rank != 1: x not a vector!"
    assert len(y.shape)==1, "rank != 1: y not a vector!"
    assert x.shape[0]==y.shape[0], "len(x) != len(y): shapes incompatible!"
    
    x=x.copy()
    y=y.copy() #ensures that original x, y not overridden
    z=0.0
    
    for i in range(x.shape[0]):
        z+=x[i]*y[i]
    
    return z

In [14]:
x=np.array([1,2,3])
y=np.array([4,5,6])

print(f"{x}.{y}=\n {naive_vector_dot(x, y)}")

[1 2 3].[4 5 6]=
 32.0


### matrix - vector product:

$$ \vec{z} = \mathbf{x} \cdot \vec{y}$$

where $\mathbf{x}$ is a matrix. 

In [15]:
def naive_matrix_vec_dot(x, y):
    assert len(x.shape)==2, "rank of x != 2: not a matrix!"
    assert len(y.shape)==1, "rank of y != 1: not a vector!"
    assert x.shape[1]==y.shape[0], "x_cols != y_rows: shapes not compatible!"
    
    x=x.copy()
    y=y.copy()
    z=np.zeros(x.shape[0])
               
    for i in range(x.shape[1]):
        for j in range(x.shape[0]):
            z[i]+= x[i, j]*y[j]
    
    return z

In [16]:
x=np.array([[1,2], [3,4]])
y=np.array([2,3])

print(f"{x}.{y}=\n {naive_matrix_vec_dot(x, y)}")

[[1 2]
 [3 4]].[2 3]=
 [ 8. 18.]


better way: make use of the vector dot product and multiply row vectors of x with column y:


In [17]:
def naive_matrix_vec_dot2(x,y):
    assert len(x.shape)==2, "rank of x != 2: not a matrix!"
    assert len(y.shape)==1, "rank of y != 1: not a vector!"
    assert x.shape[1]==y.shape[0], "x_cols != y_rows: shapes not compatible!"
    
    x=x.copy()
    y=y.copy()
    z=np.zeros(x.shape[0])
    
    for i in range(x.shape[0]):
        z[i]=naive_vector_dot(x[i,:], y)
        
    return z

In [18]:
x=np.array([[1,2], [3,4]])
y=np.array([2,3])

print(f"{x}.{y}={naive_matrix_vec_dot2(x, y)}")

[[1 2]
 [3 4]].[2 3]=[ 8. 18.]


### Tensor dot product:

$$ \mathbf{z} = \mathbf{x} \cdot \mathbf{y}$$

where $\mathbf{x}$ and $\mathbf{y}$ ar shape-compatible  matrixices (i.e. `len(x_rows)=len(y_columns)`. 

In [19]:
def naive_tensor_dot_product(x, y):
    assert len(x.shape)==2, "Rank of x != 2"
    assert len(y.shape)==2, "Rank of y != 2"
    assert x.shape[0]==y.shape[1], "columns of x don't match rows of y!"
    
    x=x.copy()
    y=y.copy()
    z=np.zeros((x.shape[0], y.shape[1]))
    
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            z[i,j]=naive_vector_dot(x[i,:], y[:, j])
    
    return z

In [20]:
x=np.array([[1,2], [3,4]])
y=np.array([[1,2], [3,4]])
print(f"{x}\n.\n{y}\n=\n{naive_tensor_dot_product(x, y)}")

[[1 2]
 [3 4]]
.
[[1 2]
 [3 4]]
=
[[ 7. 10.]
 [15. 22.]]


### Shapes of tensor products

<img src="https://dpzbhybb2pdcj.cloudfront.net/chollet/Figures/02fig05.jpg" width=400>

The dot product between higher-dimensional tensors:

$$(a,b) \cdot (b,c) \rightarrow (a, c)$$


$$(a,b,c,d) \cdot (d,) \rightarrow (a, b, c)$$

$$(a,b,c,d) \cdot (d, e) \rightarrow (a, b, c, e)$$

## Geometric interpretation of deep learning

Think of it as a series of elementary tensor operations to transform representation of data into a format, where 

### Vector addition: 

$$\vec{a} + \vec{b} = \vec{c}$$

this is an addition of two tensors of rank 1: 

<img src="https://dpzbhybb2pdcj.cloudfront.net/chollet/HighResolutionFigures/figure_2-8.png" width=200>

### matrix rotation

This is dot product of a matrix (tensor of rank 2) with a vector (tensor of rank 1):


$$ \mathbf{R} \cdot \vec{x} = \vec{y}$$

where


\begin{equation}
\mathbf{R} = 
\begin{bmatrix}
\cos x & \sin x \\
-\sin x & \cos x
\end{bmatrix}
\end{equation}


\begin{equation} 
\begin{bmatrix}
\cos \theta & \sin \theta \\
-\sin \theta & \cos \theta
\end{bmatrix} 
\cdot
\begin{pmatrix}
x \\
y
\end{pmatrix}
=
\begin{pmatrix}
x' \\
y'
\end{pmatrix}
\end{equation}


Can think of neural networks of chains of geometric transformations (tensor operations) of the input data.  These transformations take place in a high-dimensional space, implemented as long, chained, series of simple individual steps. 

### A useful mental image of class decomposition by deep learning

Imagine two sheets of coloured paper (red and blue) crumpled together into a ball. The crumpled ball represents the input data. How do we unfold it to separate the individual sheets (classes of data)?  ML is figuring out the procedure of transformations that perform this separation, very much like the sequential number of individual operations you'd perform to 'uncrumple' the pieces of paper by hand, picking a corner at a time, stretching, unfolding, shearing, etc.:


<img src="https://dpzbhybb2pdcj.cloudfront.net/chollet/HighResolutionFigures/figure_2-9.png" width=600>

This is what ML is about: finding neat representations for complex, highly folded data manifolds. The complex transformation required to achieve this is broken down into  a series of simple elementary ones - similar to what you would do to unfold the crumpled pieces of paper.

## Gradient-based optimisation

Each neural layer transforms its input data according to

```python
output = relu(dot(W, input) + b)
```

Here `W` and `b` are tensors that are attributes of the layer. They are the *weights* and *trainable parameter* of the layer (also called `kernel` and `bias`), respectively.

Initially these are assigned random values. Then they are gradually adjusted (*training*) based on a feedback signal. 

Steps:

* draw a batch of training samples `x` and targets `y`
* Run network on `x` (forward pass) yo obtain `y_predict`
* compute loss from mismatch of `y_predict` and `y`
* update weights and bias in network

Last part hardest.  Could adjust one parameter, freeze all others: takes for ever.


Better approach: network operations are differentiable.  Compute **gradient of loss** with regard to network coefficients.

Steps:

* draw a batch of training samples `x` and targets `y`
* Run network on `x` (forward pass) yo obtain `y_predict`
* compute loss from mismatch of `y_predict` and `y`
* compute gradient of the loss with regard to network's parameters (a **backward pass**)
* Move the parameters a little in opposite direction from the gradient.  E.g. 
```python
W -= step * gradient
```

This is called a *mini-batch stochastic gradient descent* (minibatch SGD). Stochastic becuse each batch of data drawn at radnom.  Figure below shows SGD down a 1D loss curve (one learnable parameter)

<img src="https://dpzbhybb2pdcj.cloudfront.net/chollet/HighResolutionFigures/figure_2-11.png" width = 400>

Observations:
    
* step sizes (**learning rate**) need to be right size.  Too small: takes to long and you migth get stuck in local minimum. Too big: miss minimum.
* Variations: *True SGD*: draw single sample.  *Minibatch SGD*: draw a batch.  *Batch SGD*: run on all available data.
* Weights ```W``` operate on high-dimensional space.  Could have as many as 100000 dimensions or more.
* Variants of SGD update `W` in different ways, e.g. SGD with momentum, Adagrad, RMSProp, etc.  

<img src="https://dpzbhybb2pdcj.cloudfront.net/chollet/Figures/02fig12.jpg" width = 400>

Example of SGD down a 2D loss surface (2 learnable parameters). 

To help get you over and out of local minima, concept of *momentum* is useful. Think of ball rolling down the surface.  Momentum too small: gets stuck in local minimum.  With enough momentum gets over and out of local minumum until it hits global minimum. Base momentum on current slope value and also previous parameter update.

```python
past_velocity = 0.
momentum = 0.1 
while loss < 0.01: #Optmimisation loop
    w, loss, gradient= get_current_parameters()
    w = w + momentum * velocity - learning_rate * gradient
    past_velocity = velocity
    update_parameter(w)
   ```

In practice, eneural network function consists of may tensor operations chained together, each of which has a simple, known derivativ.  E,g, consider network $f$, composed of 3 tensor operations $\mathbf{a}$, $\mathbf{b}$, and $\mathbf{c}$ with weight matrices $\mathbf{W1}$, $\mathbf{W2}$,$\mathbf{W3}$:

$$f(\mathbf{W1, W2, W3}) = \mathbf{a} (\mathbf{W1, b}( \mathbf{W2, c} (\mathbf{W3})))$$

chain rule:

$$\frac{d}{dx}f(g(x)) = f'(g(x))*g'(x)$$

applying chain rule to neural network gives rise to an algorithm called **backpropagation**, or *reverse-mode differentiation*.  You start with final loss value and work backwards from top layers to bottom layers, applying the chain rule to compute contribution that each parameter had in loss value.  

Tensor flow can do symbolic differentiation so gradient function becomes explicitly available. This means that Backpropagation algorithm simply becomes a call to this function. 

## Summary

* Learning means finding a combination of model parameters that minimizes a loss function for a given set of training data samples and their corresponding targets.
* Learning happens by drawing random batches of data samples and their targets, and computing the gradient of the network parameters with respect to the loss on the batch. The network parameters are then moved a bit (the magnitude of the move is defined by the learning rate) in the opposite direction from the gradient.
* The entire learning process is made possible by the fact that neural networks are chains of differentiable tensor operations, and thus it’s possible to apply the chain rule of derivation to find the gradient function mapping the current parameters and current batch of data to a gradient value.
* Two key concepts you’ll see frequently in future chapters are loss and optimizers. These are the two things you need to define before you begin feeding data into a network.
* The loss is the quantity you’ll attempt to minimize during training, so it should represent a measure of success for the task you’re trying to solve.
* The optimizer specifies the exact way in which the gradient of the loss will be used to update parameters: for instance, it could be the RMSProp optimizer, SGD with momentum, and so on.